<a href="https://colab.research.google.com/github/trongtn2110/CS114.L21/blob/master/Report_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**BÁO CÁO:** sklearn hand-written digits recognition
---
##**THÀNH VIÊN:**
1. **Nguyễn Thành Trọng** - *19522410*
2. **Đào Văn Tài** - *19522148*
3. **Ngô Gia Kiệt** - *19521725*
---



#2. Yêu cầu đề bài:

  1. Yêu cầu code:
    * Tạo một notebook colab với các đoạn code và chú thích cần thiết để thực hiện công việc sau:
    >Train model cho bài toán hand-written digits recognition theo example của sklearn: https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py
    >
    >Truy cập webcam của máy tính từ colab.
    Chụp ảnh một tờ giấy đang được giơ ra trước webcam
    >
    >Nhận dạng xem chữ số đang viết trên tờ giấy là số mấy.

  2. Yêu cầu phân tích:
    * Thông qua thực nghiệm chạy thử nhiều lần trả lời các câu hỏi sau
    > 
    >Kết quả chạy thực tế với webcam của laptop và giấy viết tay của các bạn có tốt không?
    >
    >> * Kết quả chạy thực tế với webcam của laptop và giấy viết tay là chưa tốt.
    > 
    >Ủa mà nhìn vào đâu để kết luận là tốt hay không?
    >
    >> * Dựa vào số nhóm em cho model dự đoán thì tỉ lệ sai còn cao.
    >
    >Nếu không tốt thì lý do tại sao?
    >
    >> * Do khâu tiền xử lí ảnh còn chưa tốt lắm.
    >> * Do ảnh đầu vào có kích thước lớn (480x360 )mà kích thước dữ liệu training nhỏ hơn nhất nhiều (8x8) nên việc thay đổi kích thước ảnh làm mất mát dữ liệu.
    >
    >Nếu tốt thì tiếp theo có thể dùng model digits recognition này vào các bài toán phức tạp hơn như nhận dạng biển số nhà không?
    >
    >> * Theo nhóm em nghĩ nếu tốt thì chúng ta vẫn áp dụng vào các bài toán phức tạp hơn như bài nhận dạng biển số nhà vì lúc đó ta sẽ chia cắt các con số riêng biệt nhau rồi nhận dạng chúng và ghép lại đưa ra output, vì làm như vậy thì ta tránh được các trường hợp bị nhiễu bởi các chữ số gần nhau.

In [ ]:
print(__doc__)

# Author: Gael Varoquaux <gael dot varoquaux at normalesup dot org>
# License: BSD 3 clause

# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

Tập dữ liệu chữ số bao gồm hình ảnh 8x8 pixel của các chữ số.

In [ ]:
digits = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=5, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

#**QUÁ TRÌNH TRAINING:**
---

In [ ]:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)
predicted = clf.predict(X_train)

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, prediction in zip(axes, X_train, predicted):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title(f'Prediction: {prediction}')

#**LẤY ẢNH TỪ WEBCAM:**
---

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

#**NHẬN DIỆN SỐ TỪ ẢNH CỦA WEBCAM TRÊN:**
---

In [ ]:
import cv2
import numpy as npgray = cv2.imread('photo.jpg',0)
gray = cv2.resize(gray,(8,8))
gray= gray/16
gray = gray.reshape((1,-1))
pre = clf.predict(gray)
_, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
for ax, image, prediction in zip(axes, gray, pre):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title(f'Prediction: {prediction}')